# Relevant News For Netherlands Expats

A solution for having up-to-date information relevant to expats in the Netherlands. It scarps main news portals and makes a summary with relevant information.

## List of Sources
https://www.dutchnews.nl/

## Setup

In [119]:
#imports

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [126]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL_LINKS = 'gpt-5-nano'      #small/cheap model for link selection
MODEL_DIGEST = 'gpt-4.1-mini'   #better model for final digest
openai = OpenAI()

DEFAULT_MAX_CATEGORIES = 30
DEFAULT_MAX_ARTICLES_PER_CATEGORY = 100
DEFAULT_MAX_ARTICLES_TOTAL = 300

news_source = "https://www.dutchnews.nl/"

API key looks good so far


In [ ]:
links = fetch_website_links("https://www.dutchnews.nl")
links

## Prompts

In [128]:
category_link_system_prompt = """
You are provided with a list of links found on the homepage of a Netherlands news website.
Your task is to pick links that are *news category/section* pages (e.g. politics, economy, housing, health, migration, transport, culture).
Exclude: privacy/terms, about, contact, subscribe, login, donate, ads, newsletters, social media, PDFs, mailto links.

Return JSON in this exact shape:

{
  "categories": [
    {"name": "politics", "url": "https://full.url/to/category"},
    {"name": "economy", "url": "https://full.url/to/category"}
  ]
}

Rules:
- URLs must be full absolute https URLs.
- Prefer a diverse set of categories (not 6 variations of the same thing).
- If the site is in Dutch, still return category names in English.
"""

article_link_system_prompt = """
You are provided with a list of links found on a *category page* of a Netherlands news website.
Pick links that are *individual news articles* from that category.

Return UP TO {max_articles} articles. If there are enough, return as close to {max_articles} as possible.
Prefer the most recent-looking ones. Always include housing market news.
Exclude: category index pages, tag pages, author pages, pagination, subscribe/login, privacy/terms, newsletters, social media.

Return JSON in this exact shape:

{
  "articles": [
    {"title": "short title", "url": "https://full.url/to/article"},
    {"title": "short title", "url": "https://full.url/to/article"}
  ]
}

Rules:
- URLs must be full absolute https URLs.
- Titles should be short; if the title isn't obvious from the URL, make a best guess.
"""

digest_system_prompt = """
You are an assistant creating a practical news digest for expats living in the Netherlands. Your ton of voice a bit sarcastic and straightforward and can remind Mark Manson style.
You will be given scraped text from several recent news articles.

Write in English, in markdown WITHOUT code blocks.

Requirements:
- Start with a short "What were important events this week" section (3-7 bullets).
- Then group items by theme 5-7 for each gourp (e.g. migration/IND, taxes/benefits, housing, transport, healthcare, safety, education, politics/economy).
- For each item: 1-2 sentence summary + "why it's important for expats" + (optional) "what you can do" if there is a practical action.
- At the end of each group include "Sources" as a bullet list of URLs used.
- If the scraped text is incomplete (paywall/JS), say so and avoid overconfident claims.
"""

## Step 1: category selection

In [112]:
def get_category_links_user_prompt(url):
    links = fetch_website_links(url)

    user_prompt = f"""
Here is the list of links on the website {url} -
Pick the category/section pages.

Links (some might be relative links):
"""
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
print(get_category_links_user_prompt("https://www.dutchnews.nl"))

In [125]:
def selec_category_links(url, max_categories=DEFAULT_MAX_CATEGORIES):
    print(f"Selecting relevant category links for {url} by calling {MODEL_LINKS}")
    response = openai.chat.completions.create(
        model=MODEL_LINKS,
        messages=[
            {"role": "system", "content": category_link_system_prompt},
            {"role": "user", "content": get_category_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    
    data = json.loads(response.choices[0].message.content)

    categories = data.get("categories", [])
    if not isinstance(categories, list):
        categories = []

    print(f"Found {len(categories)} relevant category links")

    data["categories"] = categories[:max_categories]
    return data

In [ ]:
selec_category_links("https://www.dutchnews.nl")

## Step 2: article selection per category

In [120]:

def get_article_links_user_prompt(category_url, max_links=300):
    links = fetch_website_links(category_url)

    links = [str(l).strip() for l in links if l]

    links = list[str](dict.fromkeys(links))
    links = links[:max_links]
    user_prompt = f"""
Here is the list of links found on the category page {category_url}

Pick recent article links. 
Links (some might be relative links):
"""
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
print(get_article_links_user_prompt("https://www.dutchnews.nl/category/economy/"))

In [115]:
def select_recent_articles(category_url, max_articles=DEFAULT_MAX_ARTICLES_PER_CATEGORY):
    response = openai.chat.completions.create(
        model=MODEL_LINKS,
        messages=[
            {"role": "system", "content": article_link_system_prompt},
            {"role": "user", "content": get_article_links_user_prompt(category_url)},
        ],
        response_format={"type": "json_object"},
    )
    data = json.loads(response.choices[0].message.content)
    articles = data.get("articles", [])

    result = []
    for a in articles:
        url = a.get("url")
        title = a.get("title")
        if not url:
            continue
        result.append({"title": title, "url": url})
    return result[:max_articles]

In [ ]:
print(select_recent_articles("https://www.dutchnews.nl/category/economy/"))

## Step 3: fetch + aggregate text

In [ ]:
def fetch_homepage_categories_and_articles(
    home_url,
    max_categories=DEFAULT_MAX_CATEGORIES,
    max_articles_per_category=DEFAULT_MAX_ARTICLES_PER_CATEGORY,
    max_articles_total=DEFAULT_MAX_ARTICLES_TOTAL,
):

    homepage = fetch_website_contents(home_url)
    categories_data = selec_category_links(home_url, max_categories=max_categories)
    categories = categories_data.get("categories", [])

    #Pick articles
    all_articles = []
    for cat in categories:
        cat_url = cat.get("url")
        cat_name = cat.get("name", "unknown")

        picked = select_recent_articles(cat_url, max_articles=max_articles_per_category)
        for a in picked:
            all_articles.append({"category": cat_name, **a})
        if len(all_articles) >= max_articles_total:
            break
    all_articles = all_articles[:max_articles_total]

# Fetch articles contents
    result = f"## Homepage ({home_url})\n\n{homepage}\n\n## Articles\n"
    for a in all_articles:
        result += f"\n\n###Category: {a['category']}\n"
        result += f"### Title (from link selection): {a['title']}\n"
        result += f"### URL: {a['url']}\n\n"
        result += fetch_website_contents(a["url"])

# Keep input reasonably bounded
    return result[:38_000], [a["url"] for a in all_articles]

def get_digest_user_prompt(home_url, **kwargs):
    blob_text, urls = fetch_homepage_categories_and_articles(home_url, **kwargs)
    user_prompt = f"""
You are creating a digest for expats based on these scraped pages.
The data below may contain navigation text; focus on actual news content.

Scraped data:
{blob_text}

All article URLs (for reference):
{chr(10).join(urls)}

Remember to include a "Sources" at the end of each section.
"""
    return user_prompt



## Step 4: final digest

In [117]:
def create_expat_digest(home_url, **kwargs):
    prompt = get_digest_user_prompt(home_url, **kwargs)
    response = openai.chat.completions.create(
        model=MODEL_DIGEST,
        messages=[
            {"role": "system", "content": digest_system_prompt},
            {"role": "user", "content": prompt},
        ],
    )
    return display(Markdown(response.choices[0].message.content))

def stream_expat_digest(home_url, **kwargs):
    prompt = get_digest_user_prompt(home_url, **kwargs)
    stream = openai.chat.completions.create(
        model=MODEL_DIGEST,
        messages=[
            {"role": "system", "content": digest_system_prompt},
            {"role": "user", "content": prompt},
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)


    

## Try it out

In [124]:
stream_expat_digest(news_source)

Selecting relevant category links for https://www.dutchnews.nl/ by calling gpt-5-nano
Found 10 relevant links


# This Week in the Netherlands for Expats (Jan 2026)

### What were important events this week?
- House price increases are slowing down, but still expected to rise 3% in 2026.
- More young first-time buyers are snapping up houses as private landlords sell off rental properties.
- A minority government cabinet has been formed by D66, CDA, and VVD, meaning lots of political horse-trading ahead.
- Local councils want Schiphol Airport to ban night flights to protect residents’ health.
- Inflation dipped slightly to 2.8%, still riding above the eurozone average.
- Fewer companies are going bust this year, with a 15% decrease in bankruptcies.
- Cafés and restaurants see higher spending, but many owners still struggle under debt and staff shortages.

---

## Housing & Real Estate

**House price rise levels off with predicted 3% increase in 2026**  
House prices surged almost 8% in Q3 2025 but are now leveling off, with economists forecasting a modest 3% rise this year and 4% next year. Newly built homes are pricier (€523,000) than existing ones (€487,000). Plans to phase out mortgage interest tax relief will barely affect prices, thanks to offsetting income tax cuts.  
*Why it matters*: If you're looking to buy, the market might cool slightly but prices are still going up. If you're renting, expect more turnover as landlords sell off properties due to new rent control laws.  
*What you can do*: Consider buying sooner rather than later if you're a first-timer, as prices won't be dropping any time soon. Landlords selling means more houses on the market for buyers.  

**More houses sold to young first-time buyers as landlords sell up**  
Young buyers now account for over half of home sales, with 11% under 25 years old (up from 4%). Landlords are offloading homes thanks to stricter rent regulations, increasing the cheaper housing supply. However, many young buyers rely on parental support and sometimes skip thorough checks on property condition.  
*Why it matters*: It's slightly easier for young expats to enter the housing market, but be cautious: buying a home without proper inspections is a recipe for expensive surprises.  
*What you can do*: If you're a first-time buyer, get a proper survey before signing anything and don't just rely on a parent's financial help. Cheap houses are coming but may come with hidden issues.  

**Sources:**  
- https://www.dutchnews.nl/2026/01/house-price-rise-levels-off-economists-see-3-increase-in-2026/  
- https://www.dutchnews.nl/2026/01/more-houses-sold-to-young-first-time-buyers-as-landlords-sell-up/  

---

## Economy & Business

**Spending in cafés rises but financial troubles persist**  
Better purchasing power has led to 5% higher turnover in restaurants and cafés last year, though nearly a quarter of small owners with debts say their financial situation is serious. The sector still battles high debts, fluctuating food costs, and staff shortages, causing more closures than in the past decade.  
*Why it matters*: Enjoying your coffee or a night out is fine, but behind the scenes, the hospitality industry remains fragile in the Netherlands.  
*What you can do*: Support your local spots but expect occasional closures or price rises due to the difficult operating environment.  

**Bankruptcies down 15% in 2025**  
The number of company bankruptcies decreased by 15% to 3,636 last year, a positive sign post-pandemic and crisis periods. Retail and trade sectors had the highest failures. The rate is far below the peak during the euro crisis and shows slowly improving business resilience.  
*Why it matters*: A more stable business climate can be good for job security and economic confidence for expats working or investing here.  

**Inflation slips slightly to 2.8%, still above eurozone average**  
Inflation dropped marginally in December but remains elevated at 2.8%, compared to the eurozone average of 2.1%. Energy costs eased somewhat, but food prices keep inching up by around 3%.  
*Why it matters*: Your groceries and bills might not get cheaper soon, so budget wisely.  

**Sources:**  
- https://www.dutchnews.nl/2026/01/spending-in-cafes-goes-up-but-financial-problems-remain/  
- https://www.dutchnews.nl/2026/01/fewer-firms-go-bust-as-bankruptcy-rate-goes-down-15/  
- https://www.dutchnews.nl/2026/01/inflation-falls-slightly-to-2-8-still-above-eurozone-average/  

---

## Politics & Government

**D66, CDA, and VVD form minority cabinet**  
The three largest parties in the October election agreed to govern as a minority cabinet, holding 66 out of 150 parliamentary seats, meaning they’ll need support from other parties on a case-by-case basis. D66's Rob Jetten will be the prime minister. This political setup is rare and promises lots of compromises and potentially shaky governance.  
*Why it matters*: Policy changes may be slower and messier; prepare for a political dance that could affect everything from housing regulations to taxes.  

**Councils demand Schiphol Airport night flight ban for health reasons**  
Fifteen local governments near Schiphol want the airport to shut down flights between 11 pm and 7 am, arguing that nighttime air traffic disrupts sleep and harms residents' health. Schiphol sees 2,400+ night flights annually and has more lenient night flight rules compared to other European hubs.  
*Why it matters*: Noise disturbances could get less in some areas if this push succeeds. Less night traffic might also mean changes to your travel plans with fewer late or early flights.  
*What you can do*: If you live near Schiphol and hate noise, support local council campaigns or voice your opinion to your municipal council.  

**Sources:**  
- https://www.dutchnews.nl/2026/01/d66-cda-and-vvd-agree-to-form-a-minority-dutch-cabinet/  
- https://www.dutchnews.nl/2026/01/councils-call-for-schiphol-to-shut-at-night-to-protect-health/  

---

# Final Notes

Nothing earth-shattering this week but plenty to keep you busy if you’re deep into house hunting or enjoying a café culture fix. The politics might get thrillingly messy, and if you live near Schiphol, your sleep might finally become a political issue. Otherwise, keep an eye on inflation and the shaky hospitality sector — your wallet and your evenings depend on it.

---

Let me know if you want me to keep dissecting Dutch news — there’s always more drama lurking behind the tulips.